# 循环神经网络
:label:`sec_rnn`


在 :numref:`sec_language-model` 中，我们描述了用于语言模型的马尔可夫模型和 $n$-gram 模型，在这些模型中，时间步 $t$ 的标记 $x_t$ 的条件概率仅依赖于前 $n-1$ 个标记。
如果我们想将早于时间步 $t-(n-1)$ 的标记对 $x_t$ 的可能影响纳入考虑，
我们需要增加 $n$。
然而，随着 $n$ 的增加，模型参数的数量也会呈指数级增长，因为对于词汇表 $\mathcal{V}$ 我们需要存储 $|\mathcal{V}|^n$ 个数字。
因此，与其建模 $P(x_t \mid x_{t-1}, \ldots, x_{t-n+1})$，不如使用一个隐变量模型，

$$P(x_t \mid x_{t-1}, \ldots, x_1) \approx P(x_t \mid h_{t-1}),$$

其中 $h_{t-1}$ 是一个 *隐藏状态*，它存储到时间步 $t-1$ 为止的序列信息。
一般来说，
任何时间步 $t$ 的隐藏状态都可以基于当前输入 $x_{t}$ 和前一隐藏状态 $h_{t-1}$ 计算得出：

$$h_t = f(x_{t}, h_{t-1}).$$
:eqlabel:`eq_ht_xt`

对于 :eqref:`eq_ht_xt` 中足够强大的函数 $f$，隐变量模型不是近似。毕竟，$h_t$ 可以简单地存储它迄今为止观察到的所有数据。
然而，这可能会使计算和存储变得昂贵。

回想我们在 :numref:`chap_perceptrons` 中讨论过的具有隐藏单元的隐藏层。
值得注意的是，
隐藏层和隐藏状态指的是两个非常不同的概念。
如前所述，隐藏层是在从输入到输出路径上不可见的层。
而隐藏状态在技术上是我们在给定步骤所做的任何事情的 *输入*，并且它们只能通过查看之前时间步的数据来计算。

*循环神经网络* (RNNs) 是具有隐藏状态的神经网络。在介绍 RNN 模型之前，我们先回顾一下在 :numref:`sec_mlp` 中介绍的多层感知机 (MLP) 模型。

In [1]:
import torch
from d2l import torch as d2l

## 无隐藏状态的神经网络

让我们看一下具有单个隐藏层的多层感知机（MLP）。
设隐藏层的激活函数为 $\phi$。
给定一个大小为 $n$ 且有 $d$ 个输入的小批量样本 $\mathbf{X} \in \mathbb{R}^{n \times d}$，隐藏层输出 $\mathbf{H} \in \mathbb{R}^{n \times h}$ 计算如下：

$$\mathbf{H} = \phi(\mathbf{X} \mathbf{W}_{\textrm{xh}} + \mathbf{b}_\textrm{h})$$
:eqlabel:`rnn_h_without_state`

在 :eqref:`rnn_h_without_state` 中，我们有隐藏层的权重参数 $\mathbf{W}_{\textrm{xh}} \in \mathbb{R}^{d \times h}$、偏置参数 $\mathbf{b}_\textrm{h} \in \mathbb{R}^{1 \times h}$ 和隐藏单元数量 $h$。
这样，我们在求和时应用广播（见 :numref:`subsec_broadcasting`）。
接下来，隐藏层输出 $\mathbf{H}$ 用作输出层的输入，计算公式为

$$\mathbf{O} = \mathbf{H} \mathbf{W}_{\textrm{hq}} + \mathbf{b}_\textrm{q}$$

其中 $\mathbf{O} \in \mathbb{R}^{n \times q}$ 是输出变量，$\mathbf{W}_{\textrm{hq}} \in \mathbb{R}^{h \times q}$ 是权重参数，$\mathbf{b}_\textrm{q} \in \mathbb{R}^{1 \times q}$ 是输出层的偏置参数。如果这是一个分类问题，我们可以使用 $\mathrm{softmax}(\mathbf{O})$ 来计算输出类别的概率分布。

这完全类似于我们在 :numref:`sec_sequence` 中解决的回归问题，因此我们省略细节。
简而言之，我们可以随机选择特征-标签对并通过自动微分和随机梯度下降来学习网络参数。

## 带有隐藏状态的循环神经网络
:label:`subsec_rnn_w_hidden_states`

当我们有隐藏状态时，情况完全不同。让我们更详细地看看结构。

假设我们在时间步 $t$ 有一个小批量输入 $\mathbf{X}_t \in \mathbb{R}^{n \times d}$。
换句话说，
对于一个小批量的 $n$ 个序列示例，
$\mathbf{X}_t$ 的每一行对应于序列中时间步 $t$ 的一个示例。
接下来，
令 $\mathbf{H}_t  \in \mathbb{R}^{n \times h}$ 表示时间步 $t$ 的隐藏层输出。
与 MLP 不同，这里我们保存了上一个时间步的隐藏层输出 $\mathbf{H}_{t-1}$ 并引入了一个新的权重参数 $\mathbf{W}_{\textrm{hh}} \in \mathbb{R}^{h \times h}$ 来描述如何在当前时间步使用上一个时间步的隐藏层输出。具体来说，当前时间步的隐藏层输出由当前时间步的输入和上一个时间步的隐藏层输出共同决定：

$$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{\textrm{xh}} + \mathbf{H}_{t-1} \mathbf{W}_{\textrm{hh}}  + \mathbf{b}_\textrm{h})$$
:eqlabel:`rnn_h_with_state`

与 :eqref:`rnn_h_without_state` 相比，:eqref:`rnn_h_with_state` 多了一个项 $\mathbf{H}_{t-1} \mathbf{W}_{\textrm{hh}}$ 因此
实例化了 :eqref:`eq_ht_xt`。
从相邻时间步的隐藏层输出 $\mathbf{H}_t$ 和 $\mathbf{H}_{t-1}$ 之间的关系，
我们知道这些变量捕获并保留了序列到当前时间步的历史信息，就像神经网络当前时间步的状态或记忆一样。因此，这样的隐藏层输出称为 *隐藏状态*。
由于隐藏状态在当前时间步使用了上一个时间步的相同定义，:eqref:`rnn_h_with_state` 的计算是 *递归* 的。因此，正如我们所说，基于递归计算的带有隐藏状态的神经网络被称为 *循环神经网络*。
在 RNN 中执行
:eqref:`rnn_h_with_state`
计算的层称为 *循环层*。

构建 RNN 有许多不同的方法。
那些通过 :eqref:`rnn_h_with_state` 定义隐藏状态的方法非常常见。
对于时间步 $t$，
输出层的输出类似于 MLP 中的计算：

$$\mathbf{O}_t = \mathbf{H}_t \mathbf{W}_{\textrm{hq}} + \mathbf{b}_\textrm{q}$$

RNN 的参数
包括隐藏层的权重 $\mathbf{W}_{\textrm{xh}} \in \mathbb{R}^{d \times h}, \mathbf{W}_{\textrm{hh}} \in \mathbb{R}^{h \times h}$，
以及偏置 $\mathbf{b}_\textrm{h} \in \mathbb{R}^{1 \times h}$，
还有输出层的权重 $\mathbf{W}_{\textrm{hq}} \in \mathbb{R}^{h \times q}$
和偏置 $\mathbf{b}_\textrm{q} \in \mathbb{R}^{1 \times q}$。
值得一提的是，
即使在不同的时间步，
RNN 始终使用这些模型参数。
因此，RNN 的参数化成本不会随着时间步数的增加而增长。

:numref:`fig_rnn` 描述了 RNN 在三个相邻时间步的计算逻辑。
在任何时间步 $t$，
隐藏状态的计算可以视为：
(i) 将当前时间步 $t$ 的输入 $\mathbf{X}_t$ 与上一个时间步 $t-1$ 的隐藏状态 $\mathbf{H}_{t-1}$ 拼接；
(ii) 将拼接结果送入具有激活函数 $\phi$ 的全连接层。
该全连接层的输出是当前时间步 $t$ 的隐藏状态 $\mathbf{H}_t$。
在这种情况下，
模型参数是 :eqref:`rnn_h_with_state` 中的 $\mathbf{W}_{\textrm{xh}}$ 和 $\mathbf{W}_{\textrm{hh}}$ 的拼接，以及偏置 $\mathbf{b}_\textrm{h}$。
当前时间步 $t$ 的隐藏状态 $\mathbf{H}_t$ 将参与计算下一个时间步 $t+1$ 的隐藏状态 $\mathbf{H}_{t+1}$。
此外，$\mathbf{H}_t$ 还将被送入全连接输出层
以计算当前时间步 $t$ 的输出
$\mathbf{O}_t$。

![带有隐藏状态的 RNN。](../img/rnn.svg)
:label:`fig_rnn`

我们刚刚提到，隐藏状态的计算 $\mathbf{X}_t \mathbf{W}_{\textrm{xh}} + \mathbf{H}_{t-1} \mathbf{W}_{\textrm{hh}}$ 等价于
$\mathbf{X}_t$ 和 $\mathbf{H}_{t-1}$ 的拼接
与
$\mathbf{W}_{\textrm{xh}}$ 和 $\mathbf{W}_{\textrm{hh}}$ 的拼接
的矩阵乘法。
虽然这可以通过数学证明，
但下面我们将仅使用一个简单的代码片段作为演示。
首先，
我们定义矩阵 `X`、`W_xh`、`H` 和 `W_hh`，其形状分别为 (3, 1)、(1, 4)、(3, 4) 和 (4, 4)。
将 `X` 乘以 `W_xh`，将 `H` 乘以 `W_hh`，然后将这两个乘积相加，
我们得到一个形状为 (3, 4) 的矩阵。

In [2]:
X, W_xh = torch.randn(3, 1), torch.randn(1, 4)
H, W_hh = torch.randn(3, 4), torch.randn(4, 4)
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 1.2526,  0.0580, -3.3460, -0.2519],
        [-1.3064,  1.4132, -0.1435,  0.3482],
        [ 3.1495,  0.8172,  1.5167, -0.9038]])

现在我们将矩阵`X`和`H`沿列（轴1）连接，将矩阵`W_xh`和`W_hh`沿行（轴0）连接。这两个连接分别得到形状为(3, 5)和(5, 4)的矩阵。将这两个连接后的矩阵相乘，我们得到与上面相同的形状为(3, 4)的输出矩阵。

In [3]:
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[ 1.2526,  0.0580, -3.3460, -0.2519],
        [-1.3064,  1.4132, -0.1435,  0.3482],
        [ 3.1495,  0.8172,  1.5167, -0.9038]])

## 基于RNN的字符级语言模型

回想在:numref:`sec_language-model`中，我们的目标是根据当前和过去的词元来预测下一个词元；因此我们将原始序列向后移动一个词元作为目标（标签）。:citet:`Bengio.Ducharme.Vincent.ea.2003`首次提出使用神经网络进行语言建模。下面我们将展示如何使用RNN构建语言模型。设小批量大小为1，文本序列为"machine"。为了简化后续章节中的训练，我们将文本分词成字符而不是单词，并考虑*字符级语言模型*。:numref:`fig_rnn_train`展示了如何通过RNN基于当前和之前的字符来预测下一个字符以实现字符级语言建模。

![基于RNN的字符级语言模型。输入序列和目标序列分别是"machin"和"achine"。](../img/rnn-train.svg)
:label:`fig_rnn_train`

在训练过程中，我们对每个时间步输出层的输出执行softmax操作，然后使用交叉熵损失计算模型输出与目标之间的误差。由于隐藏层中隐藏状态的递归计算，:numref:`fig_rnn_train`中时间步3的输出$\mathbf{O}_3$由文本序列"m"、"a"和"c"决定。因为训练数据中该序列的下一个字符是"h"，所以时间步3的损失将取决于基于特征序列"m"、"a"、"c"生成的下一个字符的概率分布以及此时间步的目标"h"。

实际上，每个词元由一个$d$维向量表示，我们使用批次大小$n>1$。因此，时间步$t$的输入$\mathbf X_t$将是一个$n\times d$矩阵，这与我们在:numref:`subsec_rnn_w_hidden_states`中讨论的内容相同。

在接下来的部分中，我们将实现用于字符级语言模型的RNN。


## 摘要

使用递归计算隐藏状态的神经网络称为循环神经网络（RNN）。
RNN的隐藏状态可以捕获序列直到当前时间步的历史信息。借助递归计算，即使时间步数增加，RNN模型参数的数量也不会增长。至于应用，RNN可用于创建字符级语言模型。


## 练习

1. 如果我们使用RNN来预测文本序列中的下一个字符，任何输出所需的维度是多少？
1. 为什么RNN能够表达某个时间步上的词元基于文本序列中所有先前词元的条件概率？
1. 如果你在一个很长的序列上反向传播，梯度会发生什么？
1. 本节描述的语言模型有哪些相关问题？

[讨论](https://discuss.d2l.ai/t/1050)